In [1]:

from torch.utils.data import DataLoader,TensorDataset
import torch.nn as nn
#import data_download
import random
import functions
import psycopg2
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset
from sklearn.model_selection import train_test_split
import torch
import polars as pl
from torch.nn import functional as F
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
dbname='ai_dataset'
dbuser='postgres'
dbpassword='parola'
dbport=5432
dbhost='127.0.0.1'
table_name='tabela'

In [3]:
tickers=["AAPL","MSFT",'SPY','XAUUSD.OANDA','BCO.ICMTRADER']
for i in range(len(tickers)):
    tickers[i]=tickers[i].replace('.','_')
net_functions=functions.Protected_execution

In [4]:
query=net_functions.create_query(tickers[0],table_name)
cursor,conn=net_functions.create_cursor(dbname,dbuser,dbpassword,dbport,dbhost)
data=pl.read_database(query=query,connection=conn)
#normalize with pandas

data=data.drop_nulls()
conn.close()
data

aapl_open,aapl_high,aapl_low,aapl_close,aapl_adj_close,aapl_volume
f64,f64,f64,f64,f64,f64
2.585,2.6696,2.5804,2.2722,2.2722,8.0723443e8
2.6832,2.7136,2.6607,2.2789,2.2789,6.196036e8
2.6725,2.675,2.6339,2.2609,2.2609,4.494224e8
2.6875,2.7393,2.6625,2.3193,2.3193,7.044576e8
2.7404,2.7571,2.705,2.3118,2.3118,6.7504077e8
…,…,…,…,…,…
195.18,195.41,192.97,193.6,193.6,3.71228e7
193.61,193.89,192.83,193.05,193.05,2.89193e7
192.49,193.5,191.09,193.15,193.15,4.80877e7


In [5]:
#get data from polars
#labels=data['labels']
train_data=data['aapl_high']
#make polars data into torch tensors
train_data=torch.tensor(train_data)

batch_size=32

In [6]:
#shorten the tensor
shortened_tensor=net_functions.tensor_shortner(train_data, batch_size)
#create batches of 32
image_tensor=net_functions.image_builder(shortened_tensor,batch_size=batch_size)
#create tensor
labels=net_functions.change(image_tensor, batch_size)

#cut the last image from data variable to adjust for change function
image_tensor=image_tensor[:-1]

In [7]:
#make labels and data into tensors
labels=torch.tensor(labels,dtype=torch.int32)
image_tensor=torch.tensor(image_tensor)

#move them to gpu
labels=labels.to(device)
image_tensor=image_tensor.to(device)


In [8]:
tensor_cpu = torch.tensor([1.0, 2.0, 3.0])
tensor_cpu=tensor_cpu.to('cuda')
tensor_cpu

tensor([1., 2., 3.], device='cuda:0')

In [9]:
def make_tensor(polars_fw,tensor, name, batch_size=32):
    #make it compatible with the tickers list
    name=name.lower()
    replacement_map=[f'{name}_low',f'{name}_adj_close',f'{name}_volume']
    for name in replacement_map:
        #make it go trough all of the steps the original tensor goes
        concat=polars[name]
        concat=self.tensor_shortner(concat, batch_size)
        concat=self.image_builder(concat, batch_size)
        concat=concat[:-1]
        concat=torch.tensor(concat)
        if tensor.device==torch.device('cuda'):
            concat=concat.to('cuda')
        tensor=torch.cat((tensor, concat),dim=1)
    return tensor
    

In [10]:
#net_functions.make_tensor(data,image_tensor,tickers[0])

In [11]:
class Protected_execution:
    def __init__(self):
        self.garbage='nothing'
    def create_query(name:str, table_name:str):
        replacement_map=[f'{name}_Open',f'{name}_High',f'{name}_Low',f'{name}_Close',f'{name}_Adj_Close',f'{name}_Volume']
        query=f"SELECT {', '.join(replacement_map)} FROM {table_name} WHERE {replacement_map[0]} IS NOT NULL ORDER BY t ASC"
        return query

    '''def request_data(self,name,table_name,dbname,dbuser,dbpassword,dbhost,dbport):
        conn=psycopg2.connect(dbname=dbname,user=dbuser,password=dbpassword,host=dbhost,port=dbport)
        cursor=conn.cursor()
        query=self.create_query(name,table_name)
        cursor.execute(query)
        data=cursor.fetchall()
        conn.close()

        return data'''


    '''def normal(arr):
        """
        Normalizes a 1D NumPy array to a range between 0 and 1.

        Args:
            arr: A 1D NumPy array containing the data to be normalized.

        Returns:
            A NumPy array containing the normalized data between 0 and 1.
        """
        min_val = np.min(arr)
        max_val = np.max(arr)
        return (arr - min_val) / (max_val - min_val)'''


    
    
    '''def ema(self,data:np.ndarray,perioada:int)->np.ndarray:
        """calculeaza Exponantial Moving aAverage"""
        ema=[data[0]]
        for i in range(1, len(data)):
            weight=2.718/(perioada+1)
            ema.append(data[i]*weight+ema[i-1]*(1-weight))
        return np.array(ema,dtype=np.float64)'''
    '''def macd(self,data:np.ndarray,fast_ema=12,slow_ema=26,signal_ema=7):
        """calculeaza Moving Avarege Convargence/Divergence"""
        short_ema=self.ema(data,fast_ema)
        long_ema=self.ema(data,slow_ema)
        macd=short_ema-long_ema
        signal=self.ema(macd,signal_ema)
        return macd[-1]-signal[-1]'''
    def insert_column(self,data,column_name,table_name,dbname,dbuser,dbpassword,dbhost,dbport):
        conn=psycopg2.connect(dbname=dbname,user=dbuser,password=dbpassword,host=dbhost,port=dbport)
        cursor=conn.cursor()
        query=f"""
            INSERT INTO {table_name} ({column_name})
            VALUES({[number for number in data]})
        """
        cursor.execute(query)
        conn.close()
    def create_cursor(dbname,dbuser,dbpassword,dbport,dbhost):
        conn=psycopg2.connect(dbname=dbname,user=dbuser,password=dbpassword,host=dbhost,port=dbport)
        cursor=conn.cursor
        return cursor,conn
    def fit_to_range_tensor(data, min_value=-5, max_value=5):
        """
        Fits a PyTorch tensor of numbers into a specified range.

        Args:
            data: The PyTorch tensor to fit.
            min_value: The minimum value in the desired range (inclusive).
            max_value: The maximum value in the desired range (inclusive).

        Returns:
            A new PyTorch tensor containing the fitted values.
        """
        if not data.numel():
            return torch.tensor([])  # Handle empty tensor

        # Find minimum and maximum values in the tensor
        data_min = torch.min(data)
        data_max = torch.max(data)

        # Handle constant data case (all elements have the same value)
        if data_min == data_max:
            return torch.full_like(data, min_value)

        # Calculate scaling factor
        scale = (max_value - min_value) / (data_max - data_min)

        # Fit the tensor using broadcasting
        fitted_data = min_value + scale * (data - data_min)

        # Clip values to the range (optional)
        clipped_data = torch.clamp(fitted_data, min_value, max_value)

        return clipped_data
    def tensor_shortner(tensor, requested_size):
        if isinstance(tensor, torch.Tensor):
            tensor_size=tensor.size()[0]
        elif isinstance(tensor, list):
            tensor_size=len(tensor)
        if tensor_size<requested_size:
            raise Exception("Requested size is bigger or equal to tensor")
        
        i=tensor_size
        while i!=0:
            if i % requested_size==0:
                return tensor[:i]
            i-=1
    def image_builder(tensor:torch.Tensor, batch_size:int,shuffle=False):
        #makes consecutive batches in form of a list
        tensor_size=tensor.size()[0]
        batches=[]
        for i in range(tensor_size):
            if i+batch_size<tensor_size-1:
                batches.append([tensor[i:i+batch_size].tolist()])
        if shuffle:
            random.shuffle(batches)
            
        return batches

    
    def change(tensor:torch.Tensor, batch_size=32):
        #shape from image_builder function goes like this:
        #[batch number][leftover dimmension][element from batch]
        empty_list=[]
        for i in range(len(tensor)-1):
            first=tensor[i+1][0][0]
            last=tensor[i][0][batch_size-1]
            diffrence=last-first
            average=(last+first)/2
            percent_diffrence=(diffrence/average)*100
            #limits to 25 points +/- to make training achivable
            #times 4 to increase sensitivity
            if percent_diffrence >20:
                percent_diffrence=20
            elif percent_diffrence<-20:
                percent_diffrence=-20
            empty_list.append(round(percent_diffrence))
        return empty_list        


    def make_tensor(self,polars_fw,tensor, name, batch_size=32):
        #make it compatible with the tickers list
        self.name=name.lower()
        self.replacement_map=[f'{self.name}_low',f'{self.name}_adj_close',f'{self.name}_volume']
        for name in self.replacement_map:
            #make it go trough all of the steps the original tensor goes
            self.concat=polars_fw[name]
            self.concat=self.tensor_shortner(self.concat, batch_size)
            self.concat=image_builder(concat, batch_size)
            self.concat=concat[:-1]
            self.concat=torch.tensor(concat)
            #move to the same device as original tensor
            if tensor.device==torch.device('cuda'):
                self.concat=self.concat.to('cuda')
            #concatanate on dim=1
            tensor=torch.cat((tensor, self.concat),dim=1)
        return tensor

In [12]:
obiect=Protected_execution()

In [14]:
x=net_functions.make_tensor(polars_fw=data, tensor=image_tensor, name=tickers[0])

In [15]:
x.shape

torch.Size([4478, 4, 32])

In [16]:
x[:][0]

tensor([[2.6696e+00, 2.7136e+00, 2.6750e+00, 2.7393e+00, 2.7571e+00, 2.9246e+00,
         3.0286e+00, 3.0857e+00, 3.0718e+00, 3.0850e+00, 3.0018e+00, 2.9164e+00,
         2.8586e+00, 2.8414e+00, 2.8364e+00, 2.7679e+00, 2.6939e+00, 2.6286e+00,
         2.7357e+00, 2.7264e+00, 2.7307e+00, 2.6914e+00, 2.5996e+00, 2.5896e+00,
         2.4814e+00, 2.4671e+00, 2.4725e+00, 2.4168e+00, 2.3839e+00, 2.4321e+00,
         2.4864e+00, 2.5361e+00],
        [2.5804e+00, 2.6607e+00, 2.6339e+00, 2.6625e+00, 2.7050e+00, 2.7082e+00,
         2.9496e+00, 2.9864e+00, 3.0214e+00, 2.9954e+00, 2.9232e+00, 2.8121e+00,
         2.7082e+00, 2.7143e+00, 2.7061e+00, 2.6161e+00, 2.5689e+00, 2.5393e+00,
         2.5311e+00, 2.6339e+00, 2.6657e+00, 2.5732e+00, 2.5371e+00, 2.3836e+00,
         2.3814e+00, 2.3571e+00, 2.3046e+00, 2.2464e+00, 2.3086e+00, 2.3214e+00,
         2.3839e+00, 2.4814e+00],
        [2.2722e+00, 2.2789e+00, 2.2609e+00, 2.3193e+00, 2.3118e+00, 2.4580e+00,
         2.5503e+00, 2.5622e+00, 2.6017e+